In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 1. Menyiapkan Augmentasi Data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

# 2. Memuat Data
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/SC/ribuan/latih',  # Path data pelatihan
    target_size=(224, 224),
    batch_size=32,  # Sesuaikan batch size
    class_mode='sparse',
    shuffle=True  # Data dilatih dengan shuffle
)

validation_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/SC/ribuan/validasi',  # Path data validasi
    target_size=(224, 224),
    batch_size=32,  # Sesuaikan batch size
    class_mode='sparse',
    shuffle=False  # Validasi tidak perlu shuffle
)

# 3. Membuat Model dengan Transfer Learning (MobileNetV2)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Membekukan beberapa layer pertama base model (misalnya sampai layer ke-100)
for layer in base_model.layers[:100]:
    layer.trainable = False

# Membebaskan beberapa layer terakhir untuk fine-tuning
for layer in base_model.layers[100:]:
    layer.trainable = True

# Membangun model dengan lapisan tambahan
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)  # Tambahkan regularisasi L2
x = Dropout(0.5)(x)  # Regularisasi dropout
x = Dense(10, activation='softmax')(x)  # Sesuaikan dengan jumlah kelas

# Final model
model = Model(inputs=base_model.input, outputs=x)

# 4. Kompilasi Model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Menampilkan ringkasan model
model.summary()

# 5. Melatih Model
# 5. Melatih Model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10  # Jumlah epoch tetap
)

# Menyimpan Model
# Menentukan versi model
version = 7
model_filename = f'model_v{version}.h5'

# Menyimpan model
model.save(model_filename)
print(f"Model disimpan ke: {model_filename}")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# 1. Mengevaluasi Model dengan Dataset Validasi
val_loss, val_accuracy = model.evaluate(validation_generator, verbose=1)
print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy:.4f}")

# 2. Mendapatkan Prediksi dan Label Asli
y_true = validation_generator.classes  # Label sebenarnya
y_pred_prob = model.predict(validation_generator, verbose=1)  # Prediksi probabilitas
y_pred = np.argmax(y_pred_prob, axis=1)  # Mengambil kelas dengan probabilitas tertinggi

# 3. Menghitung Confusion Matrix
cm = confusion_matrix(y_true, y_pred)

# 4. Menampilkan Confusion Matrix Berwarna
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=validation_generator.class_indices.keys(), yticklabels=validation_generator.class_indices.keys())
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

# 5. Menghasilkan Laporan Klasifikasi (Precision, Recall, F1 Score)
target_names = list(validation_generator.class_indices.keys())  # Nama kelas
report = classification_report(y_true, y_pred, target_names=target_names, digits=4)
print("Classification Report:")
print(report)

# 6. Menghitung Error Rate
error_rate = 1 - val_accuracy
print(f"Error Rate: {error_rate:.4f}")

In [ ]:
from ipywidgets import FileUpload, Image, VBox
from PIL import Image as PILImage
import io
import numpy as np
import tensorflow as tf

# Daftar kelas yang sesuai dengan model Anda
class_names = ['bat', 'butterfly', 'cat', 'cow', 'dog', 'elephant', 'horse', 'sheep', 'squirrel', 'tiger']

# Load model (gunakan model Anda sendiri)
model = tf.keras.models.load_model('/content/model_v7.h5')

# Membuat widget untuk mengupload file
upload = FileUpload(accept='image/*', multiple=False)

# Membuat widget Image untuk menampilkan gambar dengan ukuran tampilan yang disesuaikan
image_widget = Image()

# Fungsi untuk memproses gambar setelah diupload
def process_uploaded_image(change):
    # Mendapatkan file yang diupload
    img_data = next(iter(upload.value.values()))['content']

    # Membuka gambar dengan PIL
    img = PILImage.open(io.BytesIO(img_data))

    # Menampilkan gambar yang diupload pada widget
    img_resized = img.resize((400, 400))  # Mengubah ukuran gambar sesuai keinginan
    img_bytes = io.BytesIO()
    img_resized.save(img_bytes, format='PNG')
    img_bytes.seek(0)

    image_widget.value = img_bytes.read()  # Update widget image dengan gambar yang sudah diubah ukurannya
    image_widget.width = 300  # Atur lebar tampilan gambar
    image_widget.height = 300  # Atur tinggi tampilan gambar

    # Mengubah ukuran gambar sesuai input model (224x224)
    img_resized_for_model = img.resize((224, 224))

    # Mengubah gambar menjadi array numpy
    img_array = np.array(img_resized_for_model)

    # Menambah dimensi batch
    img_array = np.expand_dims(img_array, axis=0)

    # Normalisasi (sesuai dengan model yang digunakan)
    img_array = img_array / 255.0

    # Melakukan prediksi
    classes = model.predict(img_array)

    # Mengidentifikasi kelas berdasarkan prediksi
    predicted_class = np.argmax(classes, axis=1)

    # Menampilkan nama kelas prediksi
    predicted_class_name = class_names[predicted_class[0]]
    print(f'Predicted Class: {predicted_class_name}')

# Menyambungkan fungsi dengan upload event
upload.observe(process_uploaded_image, names='value')

# Tampilkan widget upload dan gambar
display(VBox([upload, image_widget]))